In [2]:
import numpy as np
import pandas as pd

# 데이터 시각화 패키지
from matplotlib import pyplot as plt
import matplotlib
import seaborn as sns
import plotly.express as px
from matplotlib.sankey import Sankey  
import altair as alt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Jupyter Notebook에서 그래프를 바로 확인할 수 있도록 설정
%matplotlib inline

# 통계 분석 및 모델링을 위한 패키지
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 머신러닝 및 데이터 분석 패키지
import sklearn.linear_model
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.cluster import DBSCAN, KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import (silhouette_samples, silhouette_score, accuracy_score,
                             confusion_matrix, ConfusionMatrixDisplay, classification_report,
                             mean_absolute_error, mean_squared_error, r2_score)
from sklearn.preprocessing import *
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


# 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (운영체제별 한글 지원)
from matplotlib import font_manager, rc

# '-' 기호가 제대로 보이도록 설정
matplotlib.rcParams['axes.unicode_minus'] = False

# 윈도우 환경: Malgun Gothic 폰트 설정
path = "c:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=path).get_name()
rc('font', family=font_name)

In [3]:
df1 = pd.read_csv('campaign_desc.csv')
df2 = pd.read_csv('campaign_table.csv')
df3 = pd.read_csv('coupon_used.csv')
df4 = pd.read_csv('coupon.csv')
df5 = pd.read_csv('customer_info.csv')
df6 = pd.read_csv('product.csv')

In [4]:
df7 = pd.read_csv('causal_data.csv')
df8 = pd.read_csv('transaction_data.csv')

In [39]:
pd.options.display.float_format = '{:,.0f}'.format

### 캠페인정보(df1)

In [15]:
df1.head()

start(day)  end(day)        Manager  Campaign_ID Campaign_Type
0         372       404   Levi Douglas            4             B
1         575       607  Amelia Foster           17             B
2         463       495  Ella Mitchell           10             B
3         412       460   Grace Lawson            8             A
4         323       369  Audrey Knight           30             A

### 캠페인테이블(df2)

In [5]:
df2.sort_values(by='Campaign_ID')

Household_ID  Campaign_ID Campaign_Type
6272           105            1             B
4583          1258            1             B
5851            93            1             B
5864           932            1             B
5022           293            1             B
...            ...          ...           ...
4908           864           30             A
6821          2002           30             A
3728          1364           30             A
4932           187           30             A
5715           319           30             A

[7208 rows x 3 columns]

In [6]:
len(df2['Household_ID'].unique())

1584

### 쿠폰 사용데이터(df3)

In [16]:
df3

Day  Coupon_Code  Campaign_ID  Household_ID
0     516  53010030033           13          2366
1     598  54200029176           18          2451
2     595  53800050033           18          1453
3     528  54100027032           13          1764
4     515  10000085429           13          2004
...   ...          ...          ...           ...
2313  533  10000089124           13           725
2314  670  51200000076           23           889
2315  601  55410000076           18           208
2316  603  51960010076           18          1146
2317  619  53010010033           18          2130

[2318 rows x 4 columns]

### 쿠폰데이터(df4)

In [11]:
df4.query('Coupon_Code == 52100026076 & Product_ID == 12385374')

Product_ID  Campaign_ID  Coupon_Code
55914    12385374           30  52100026076
78925    12385374           26  52100026076

### 고객데이터(df5)

In [6]:
df5

Household_ID  Household_Composition Household_Size  Number_of_kids  \
0            2124                      2              4               2   
1             483                      3              1               4   
2            1305                      5              2               1   
3             806                      1              2               4   
4            2084                      1              2               4   
..            ...                    ...            ...             ...   
796          2237                      3              1               4   
797          1479                      6              1               4   
798            57                      1              2               4   
799          1081                      1              2               4   
800          1536                      6              1               4   

    Marriage   Home_ownership  Household_head_Age  Income_range  
0        Yes        Homeowner                   2             5  
1    Unknown          Unknown                   4             4  
2         No           Renter                   1             1  
3        Yes        Homeowner                   3             6  
4        Yes        Homeowner                   5             8  
..       ...              ...                 ...           ...  
796  Unknown        Homeowner                   4             5  
797  Unknown  Probable Renter                   4             2  
798       No        Homeowner                   4             4  
799      Yes        Homeowner                   5             6  
800  Unknown          Unknown                   4             5  

[801 rows x 8 columns]

In [47]:
df5[df5['Household_ID'] == 1393]

Household_ID  Household_Composition Household_Size  Number_of_kids  \
125          1393                      3              1               4   

    Marriage Home_ownership  Household_head_Age  Income_range  
125       No      Homeowner                   3             9

In [79]:
store_counts = df5['Income_range'].value_counts().reset_index()
store_counts.columns = ['Income_range', 'count']
fig = px.bar(store_counts, x='Income_range', y='count', title='Income_range')
fig.show()

### 상품데이터(df6)

In [13]:
df6['Subcategory'].nunique()

308

In [16]:
df6[df6['Product_ID'] == 6544236]

Product_ID  Manufacturer    Brand         Category        Subcategory  \
33371     6544236            69  Private  MISC SALES TRAN  COUPON/MISC ITEMS   

                Product_type Curr_Size_of_Product  
33371  GASOLINE-REG UNLEADED

### 판촉데이터(df7)

In [19]:
df7.head()

Week_no display mailer  Product_ID  Store_ID
0       55       3      0    13213033       450
1       50       5      A     1362078       445
2       80       0      A      891463       443
3       39       5      0    10120701       381
4       39       0      A     1302011       400

### 거래기록데이터(df8)

In [9]:
df8

Household_ID    Basket_ID  Product_ID  Store_ID  Day  Quantity  \
0                1803  30780785930     1065887       338  252         1   
1                2299  33768622588     1073244       446  456         1   
2                 158  30202616809     7025114       343  225         1   
3                2347  42076926172     1064299       438  695         1   
4                1430  31625201009     1040197     31742  312         1   
...               ...          ...         ...       ...  ...       ...   
2595727           225  33655617762     6463742     31642  447         1   
2595728           266  40097402794      968269       367  551         1   
2595729          2364  30443976140     2041688       673  230         2   
2595730          1048  31145396229     1082185       436  280         1   
2595731          2270  28318130093     1138858       445  111         1   

         Sales_Value  Trans_time  Week_no  Disc(retail)  Disc(coupon)  \
0               8.99        1419       37         -1.00           0.0   
1               1.00        2030       66         -1.59           0.0   
2               1.50        1246       33         -0.50          -1.0   
3               2.50        1430      100         -2.89           0.0   
4               2.29        1423       45          0.00           0.0   
...              ...         ...      ...           ...           ...   
2595727         2.29        1942       65         -1.70           0.0   
2595728         1.99        2255       79          0.00           0.0   
2595729         1.50        1757       34         -0.62           0.0   
2595730         0.74        1445       41          0.00           0.0   
2595731         2.69         808       17         -0.30           0.0   

         Disc(coupon_match)  
0                       0.0  
1                       0.0  
2                       0.0  
3                       0.0  
4                       0.0  
...                     ...  
2595727                 0.0  
2595728                 0.0  
2595729                 0.0  
2595730                 0.0  
2595731                 0.0  

[2595732 rows x 12 columns]

In [8]:
# Quantity 엄청 많은 것도 존재
df8['Quantity'].sort_values(ascending=False)

2538671    89638
830197     85055
1442844    61335
772657     51912
1282355    48073
           ...  
2131943        0
869313         0
43079          0
326170         0
992954         0
Name: Quantity, Length: 2595732, dtype: int64

In [ ]:
#Quantity가 0인부분도 존재 근데 쿠폰 할인된게 있는데 이건 뭘까 ..
df8[df8['Quantity'] == 0]

Household_ID    Basket_ID  Product_ID  Store_ID  Day  Quantity  \
107              1917  33329735778     5978656     32004  431         0   
132               641  40826461597     5978656       374  610         0   
284              1146  40955815328     5978648       304  621         0   
463              1848  41366211953     1038732     31782  641         0   
635              1390  41532175179     1081867       293  654         0   
...               ...          ...         ...       ...  ...       ...   
2594730          1797  30142165398     5978648       322  220         0   
2595089          1350  34664637988      819845       337  502         0   
2595311           869  33421233013     5978648       310  437         0   
2595397          1534  41782635951     1002765       375  673         0   
2595575          2149  35726861272     5978648       309  533         0   

         Sales_Value  Trans_time  Week_no  Disc(retail)  Disc(coupon)  \
107              0.0        1139       62           0.0          -2.5   
132              0.0        1414       88           0.0          -3.0   
284              0.0        1010       89           0.0           0.0   
463              0.0        1410       92           0.0           0.0   
635              0.0         935       94           0.0           0.0   
...              ...         ...      ...           ...           ...   
2594730          0.0        1135       32           0.0           0.0   
2595089          0.0        1354       72           0.0           0.0   
2595311          0.0        1808       63           0.0           0.0   
2595397          0.0        1022       97           0.0           0.0   
2595575          0.0        1751       77           0.0           0.0   

         Disc(coupon_match)  
107                     0.0  
132                     0.0  
284                     0.0  
463                     0.0  
635                     0.0  
...                     ...  
2594730                 0.0  
2595089                 0.0  
2595311                 0.0  
2595397                 0.0  
2595575                 0.0  

[14466 rows x 12 columns]

In [5]:
# 일단 merge를 하고 진행을 해봐야겠다. product랑
df68 = pd.merge(df8,df6,on='Product_ID', how='inner')
df68

Household_ID    Basket_ID  Product_ID  Store_ID  Day  Quantity  \
0                1803  30780785930     1065887       338  252         1   
1                1388  30253393814     1065887       375  228         1   
2                1268  32269557669     1065887     31742  362         1   
3                 868  33944722714     1065887       323  467         1   
4                2374  40727257930     1065887       367  602         1   
...               ...          ...         ...       ...  ...       ...   
2595727          2224  30579241929      928870       315  236         1   
2595728          2184  28344932057     9803768       309  114         1   
2595729            57  31344122693     6039807       298  293         1   
2595730          1835  27469178389     1037872       375   41         1   
2595731          1409  28856455232     9423802      3313  145         1   

         Sales_Value  Trans_time  Week_no  Disc(retail)  Disc(coupon)  \
0               8.99        1419       37         -1.00           0.0   
1               9.99        1748       33          0.00           0.0   
2               9.99        2014       52          0.00           0.0   
3               7.99        1829       67         -2.00           0.0   
4               9.99        1350       87          0.00           0.0   
...              ...         ...      ...           ...           ...   
2595727         4.50        1813       34          0.00           0.0   
2595728         7.99        2119       17          0.00           0.0   
2595729         6.04        1845       43          0.00           0.0   
2595730         6.99        1830        7          0.00           0.0   
2595731         1.67         929       21         -0.32           0.0   

         Disc(coupon_match)  Manufacturer     Brand   Category  \
0                       0.0            69   Private    DRUG GM   
1                       0.0            69   Private    DRUG GM   
2                       0.0            69   Private    DRUG GM   
3                       0.0            69   Private    DRUG GM   
4                       0.0            69   Private    DRUG GM   
...                     ...           ...       ...        ...   
2595727                 0.0           504  National    DRUG GM   
2595728                 0.0          5014  National  COSMETICS   
2595729                 0.0          5016  National  COSMETICS   
2595730                 0.0          2239  National    DRUG GM   
2595731                 0.0          6351  National    GROCERY   

                  Subcategory                Product_type Curr_Size_of_Product  
0                  ANALGESICS            ADULT ANALGESICS                       
1                  ANALGESICS            ADULT ANALGESICS                       
2                  ANALGESICS            ADULT ANALGESICS                       
3                  ANALGESICS            ADULT ANALGESICS                       
4                  ANALGESICS            ADULT ANALGESICS                       
...                       ...                         ...                  ...  
2595727              MAGAZINE         AUTOMOBILE-MAGAZINE                       
2595728  MAKEUP AND TREATMENT  FACE MAKE UP AND TREATMENT                       
2595729  MAKEUP AND TREATMENT                      REVLON                       
2595730                SUNTAN                      SUNTAN                18 OZ  
2595731     DRY NOODLES/PASTA               SPAGHETTI DRY              14.5 OZ  

[2595732 rows x 18 columns]

#### 매장별 분석

In [ ]:
df68['Store_ID'].value_counts()

367     75573
406     59331
356     49332
381     45268
292     45240
        ...  
256         1
610         1
86          1
949         1
1235        1
Name: Store_ID, Length: 582, dtype: int64

In [57]:
store_counts = df68['Store_ID'].value_counts().sort_index().reset_index()
store_counts.columns = ['Store_ID', 'count']

store_counts['Store_ID'] = store_counts['Store_ID'].astype(str)
store_counts = store_counts.sort_values('Store_ID')

fig = px.line(store_counts, x='Store_ID', y='count', title='Count per Store')
fig.update_layout(xaxis_tickangle=-90)
fig.show()

In [58]:
store_sales = df68.groupby('Store_ID')['Sales_Value'].sum().reset_index()
store_sales['Store_ID'] = store_sales['Store_ID'].astype(str)
store_sales = store_sales.sort_values('Store_ID')
fig = px.line(
    store_sales,
    x='Store_ID',
    y='Sales_Value',
    title='Total Sales Value per Store',
    labels={'Store_ID': 'Store ID', 'Sales_Value': 'Total Sales ($)'}
)

# x축 라벨 기울이기
fig.update_layout(xaxis_tickangle=-90)

# 그래프 보여주기
fig.show()

In [59]:
store_sales = df68.groupby('Store_ID')['Quantity'].sum().reset_index()
store_sales['Store_ID'] = store_sales['Store_ID'].astype(str)
store_sales = store_sales.sort_values('Store_ID')
fig = px.line(
    store_sales,
    x='Store_ID',
    y='Quantity',
    title='Total Quantity per Store',
)

# x축 라벨 기울이기
fig.update_layout(xaxis_tickangle=-90)

# 그래프 보여주기
fig.show()

In [76]:
store_367 = df68[df68['Store_ID'] == 367]  # 만약 숫자형이면 == 367

# 3. product별 Sales_Value 합계 계산
product_sales = store_367.groupby('Product_type')['Sales_Value'].sum().reset_index()

# 4. 매출 기준 정렬 (선택사항)
product_sales.sort_values(by = 'Sales_Value', ascending=False).head(10)

Product_type  Sales_Value
635            GASOLINE-REG UNLEADED       31,843
131              BEERALEMALT LIQUORS        5,245
1174             POPULAR 750ML WINES        4,710
1203             PREMIUM 750ML WINES        3,807
544            FLUID MILK WHITE ONLY        3,427
1442  SOFT DRINKS 12/18&15PK CAN CAR        2,703
621   FRZN SS PREMIUM ENTREES/DNRS/N        2,495
399   DAIRY CASE 100% PURE JUICE - O        2,111
1556                   TOILET TISSUE        2,048
1406                 SHREDDED CHEESE        1,910

#### 날짜별 분석

In [7]:
agg_df = df68.groupby('Day').agg(
    Transaction_Count=('Household_ID', 'count'),  
    Total_Sales_Value=('Sales_Value', 'sum'),
    Total_Quantity=('Quantity', 'sum')
).reset_index()


In [20]:
fig1 = go.Figure()
fig1.add_trace(
    go.Scatter(
        x=agg_df['Day'],
        y=agg_df['Total_Sales_Value'],
        mode='lines',
        name='Total Sales Value',
        line=dict(color='#1f77b4')
    )
)

fig1.update_layout(
    title='Day별 Sales Value 추이',
    xaxis_title='Day',
    yaxis_title='Total Sales Value'
)

fig1.show()

In [35]:
fig2 = make_subplots(specs=[[{"secondary_y": True}]])

# 거래 횟수 (왼쪽 y축)
fig2.add_trace(
    go.Scatter(
        x=agg_df['Day'],
        y=agg_df['Transaction_Count'],
        mode='lines',
        name='Transaction Count',
        line=dict(color='#9467bd')
    ),
    secondary_y=False
)

# Quantity (오른쪽 y축)
fig2.add_trace(
    go.Scatter(
        x=agg_df['Day'],
        y=agg_df['Total_Quantity'],
        mode='lines',
        name='Total Quantity',
        line=dict(color='red'),
        opacity=0.5
    ),
    secondary_y=True
)

fig2.update_layout(
    title='Day별 거래 횟수와 Quantity 추이',
    xaxis_title='Day',
    legend=dict(x=0.01, y=0.99)
)

fig2.update_yaxes(title_text="Transaction Count", secondary_y=False)
fig2.update_yaxes(title_text="Total Quantity", secondary_y=True)

fig2.show()

In [45]:
df68[df68['Day'] == 643]

Household_ID    Basket_ID  Product_ID  Store_ID  Day  Quantity  \
1453462          1795  41400285005     1104475     31782  643         1   
1865075          1795  41400285005    17168855     31782  643         1   

         Sales_Value  Trans_time  Week_no  Disc(retail)  Disc(coupon)  \
1453462            2        2342       93            -0             0   
1865075            1        2342       93            -0             0   

         Disc(coupon_match)  Manufacturer     Brand Category   Subcategory  \
1453462                   0          1755  National  GROCERY  FROZEN PIZZA   
1865075                   0           103  National  GROCERY       NEW AGE   

                         Product_type Curr_Size_of_Product  
1453462  PIZZA/SINGLE SERVE/MICROWAVE              5.51 OZ  
1865075                  ENERGY DRINK            20 OZ PET

#### 가구별 분석

In [100]:
df_target =  df68[df68['Household_ID'] == 15].groupby('Day')['Sales_Value'].sum().reset_index()

# 2. 시각화
fig = px.line(df_target, x='Day', y='Sales_Value',
              title='15번가구의 일별 매출 추이')  # 점 표시
fig.show()

In [ ]:
df_unique_days = df68[['Household_ID', 'Day']].drop_duplicates()
df_unique_days = df_unique_days.sort_values(by=['Household_ID', 'Day'])

# 2. 고객별 방문 간격 계산
df_unique_days['Visit_Interval'] = df_unique_days.groupby('Household_ID')['Day'].diff()

# 3. 고객별 평균 방문주기 + 표준편차 계산
visit_cycle_df = df_unique_days.groupby('Household_ID')['Visit_Interval'].agg(
    Avg_Visit_Interval='mean',
    Std_Visit_Interval='std'
).reset_index()

In [88]:
visit_cycle_df.rename(columns={'Avg_Visit_Interval':'평균방문주기', 'Std_Visit_Interval':'표준편차'},inplace=True)

In [97]:
visit_cycle_df[visit_cycle_df['평균방문주기'] < visit_cycle_df['표준편차']]

Household_ID  평균방문주기  표준편차
1                2      13    15
4                5      19    23
9               10     143   160
10              11      76    85
14              15      15    24
...            ...     ...   ...
2491          2492       7     8
2492          2493       6     8
2493          2494      20    24
2494          2495      32    56
2497          2498       4     6

[1267 rows x 3 columns]

In [84]:
fig = px.histogram(visit_cycle_df, x='평균방문주기',
                   nbins=30, title='고객 평균 방문주기 분포')
fig.show()

#### 100일간의 미스테리

In [119]:
daily_households = df68.groupby('Day')['Household_ID'].nunique().reset_index()
daily_households.columns = ['Day', 'Unique_Households']


# Plotly를 활용한 시각화
fig = px.line(daily_households, x='Day', y='Unique_Households',
              title='Day별 가구 수',
              labels={'Day': 'Day', 'Unique_Households': '가구 수'})
fig.show()

In [120]:
gasoline_df = df68[df68['Product_type'] == 'GASOLINE-REG UNLEADED']

daily_sales = gasoline_df.groupby('Day')['Sales_Value'].sum().reset_index()

# 3. 시각화
fig = px.line(daily_sales, x='Day', y='Sales_Value',
              title='Day별 GASOLINE 매출 추이',
              labels={'Day': 'Day', 'Sales_Value': '총 매출'})
fig.show()

In [122]:
sales_by_product_type = df68.groupby('Product_type')['Sales_Value'].sum().reset_index()
sales_by_product_type.sort_values(by='Sales_Value', ascending=False)

Product_type  Sales_Value
912            GASOLINE-REG UNLEADED      633,957
794            FLUID MILK WHITE ONLY      161,907
2052  SOFT DRINKS 12/18&15PK CAN CAR      158,188
210              BEERALEMALT LIQUORS      148,868
461                       CIGARETTES       96,394
...                              ...          ...
1333                       MISC BULK            0
913                          GAZEBOS            0
1094             INFANTS DRESSES RTW            0
1188      LICENSE APPAREL GIRLS 7-16            0
0                                               0

[2383 rows x 2 columns]